In [1]:
# importando os pacotes que serão utilizados
import numpy as np
import cv2
import pickle as pkl

In [ ]:
# importando um arquivo pickle usando a função 'open' com o bloco 'with'
# um arquivo pickle é um arquivo binário que armazena objetos python
with open("pontos.pkl", "rb") as f:
    pontos = pkl.load(f)

# imprimindo os primeiros 5 pontos do arquivo carregado
pontos[:5]

[(538, 301), (547, 251), (491, 206), (537, 147), (483, 112)]

In [4]:
# p1 é o ponto de partida e p2 é o ponto de chegada do vetor de contagem de cruzamentos
pontos_contador = {
    "p1": np.array([320, 50]),
    "p2": np.array([320, 450]),
}

In [6]:
cv2.namedWindow("Frame", cv2.WINDOW_AUTOSIZE)

# percorrendo cada ponto da coleção 'pontos' para observar os pontos do objeto 'andante'
for ponto in pontos:

    # cria um frame preto (zeros) com 480 altura X 640 largura pixels e 3 canais de cor (RGB)
    frame = np.zeros((480, 640, 3), dtype=np.uint8)

    # desenha um círculo no frame na posição do ponto (x, y) com raio 5 e cor verde
    frame = cv2.circle(frame, ponto, 5, (0, 255, 0), -1)

    # desenha uma seta no frame que representa o vetor de contagem de cruzamentos
    frame = cv2.arrowedLine(img=frame, 
                            pt1=pontos_contador["p1"], 
                            pt2=pontos_contador["p2"], 
                            color=(0, 0, 255), 
                            thickness=2)

    # exibe o frame na tela
    cv2.imshow("Frame", frame)

    # aguarda uma tecla ser pressionada. Enquanto não for pressionada, a execução do código é pausada
    tecla = cv2.waitKey(0)

    # se a tecla pressionada for 'q', o loop é interrompido e a janela é fechada
    if tecla == ord('q'):
        break

# fecha todas as janelas abertas
cv2.destroyWindow("Frame")

In [7]:
# exibe o dicionário 'pontos_contador' na tela
pontos_contador

{'p1': array([320,  50]), 'p2': array([320, 450])}

In [8]:
# cria um vetor do objeto 'andante' para criar e testar o algoritmo de checagem de cruzamento
# p1 é o ponto de partida e p2 é o ponto de chegada do vetor 'andante'
pontos_andante = {
    "p1": np.array([300, 200]),
    "p2": np.array([340, 200]),
}
pontos_andante

{'p1': array([300, 200]), 'p2': array([340, 200])}

In [9]:
# cria o frame preto (zeros) com 480 altura X 640 largura pixels e 3 canais de cor (RGB)
frame = np.zeros((480, 640, 3), dtype=np.uint8)

# escreve na tela o texto 'vetor_contador'
frame = cv2.putText(img=frame,                                  # imagem
                    text="vetor_contador",                      # texto a ser exibido
                    org=(10, 20),                               # ponto de origem do texto
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,          # tipo da fonte
                    fontScale=0.5,                              # tamanho da fonte
                    color=(0, 0, 255),                          # cor do texto (BGR)
                    thickness=2)                                # espessura da fonte

# desenha uma seta no frame que representa o vetor de contagem de cruzamentos
frame = cv2.arrowedLine(img=frame,                              # imagem
                        pt1=pontos_contador["p1"],          # ponto de partida
                        pt2=pontos_contador["p2"],          # ponto de chegada
                        color=(0, 0, 255),                      # cor da seta (BGR)
                        thickness=2)                            # espessura da seta

# escreve na tela o texto 'vetor_andante'
frame = cv2.putText(img=frame,                                  # imagem
                    text="vetor_andante",                # texto a ser exibido
                    org=(10, 40),                               # ponto de origem do texto
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,          # tipo da fonte
                    fontScale=0.5,                              # tamanho da fonte
                    color=(0, 255, 0),                          # cor do texto (BGR)
                    thickness=2)                                # espessura da fonte

# desenha uma seta no frame que representa o vetor do objeto 'andante'
frame = cv2.arrowedLine(img=frame,                              # imagem
                        pt1=pontos_andante["p1"],    # ponto de partida
                        pt2=pontos_andante["p2"],    # ponto de chegada
                        color=(0, 255, 0),                      # cor da seta (BGR)
                        thickness=2)                            # espessura da seta

# exibe o frame na tela
cv2.imshow("Frame", frame)

# congela a execução do código até que uma tecla seja pressionada
cv2.waitKey(0)

# fecha todas as janelas abertas
cv2.destroyAllWindows()

In [10]:
# separa os valores dos pontos 'p1' e 'p2' dos vetores 'andante' e 'contador' em variaveis separadas
# para adaptar o código em relação ao trabalho de modelagem 'pdf'

ai = pontos_andante["p1"][0]
aj = pontos_andante["p1"][1]
bi = pontos_andante["p2"][0]
bj = pontos_andante["p2"][1]

ci = pontos_contador["p1"][0]
cj = pontos_contador["p1"][1]
di = pontos_contador["p2"][0]
dj = pontos_contador["p2"][1]

In [11]:
# transformando os dicionários em vetores numpy
vetor_andante = np.array([bi - ai, bj - aj])
vetor_contador = np.array([di - ci, dj - cj])

print(vetor_andante)
print(vetor_contador)

[40  0]
[  0 400]


In [12]:
# observar a modelagem em 'pdf' para entender a lógica do cálculo
matriz_m = np.array([
    [di - ci, ai - bi],
    [dj - cj, aj - bj]
])
matriz_m

array([[  0, -40],
       [400,   0]])

In [13]:
# calculando a inversa da matriz 'm'
matriz_m_inv = np.linalg.inv(matriz_m)
matriz_m_inv

array([[ 0.    ,  0.0025],
       [-0.025 , -0.    ]])

In [14]:
# observar a modelagem em 'pdf' para entender a lógica do cálculo
vetor_ac = np.array([[ai - ci], 
                     [aj - cj]])
vetor_ac

array([[-20],
       [150]])

In [15]:
# o produto da matriz inversa 'm' com o vetor 'ac' resulta nos coeficientes 'k1' e 'k2'
k2, k1 = matriz_m_inv.dot(vetor_ac)
k1, k2

(array([0.5]), array([0.375]))

### Perceber que houve o cruzamento entre os vetores `andante` e `contador`, pois `0 <= k1 <= 1` e `0 <= k2 <= 1`

In [16]:
# importando o pacote que vai parear os pontos
from itertools import pairwise

In [17]:
# pareando os pontos
pontos_pareados = pairwise(pontos)
pontos_pareados

In [18]:
# transformando o objeto 'pairwise' em uma lista para imprimir seus elementos na tela
pontos_pareados = list(pairwise(pontos))
pontos_pareados

[((538, 301), (547, 251)),
 ((547, 251), (491, 206)),
 ((491, 206), (537, 147)),
 ((537, 147), (483, 112)),
 ((483, 112), (422, 196)),
 ((422, 196), (481, 278)),
 ((481, 278), (394, 348)),
 ((394, 348), (349, 269)),
 ((349, 269), (450, 224)),
 ((450, 224), (371, 176)),
 ((371, 176), (257, 127)),
 ((257, 127), (185, 176)),
 ((185, 176), (127, 255)),
 ((127, 255), (198, 353)),
 ((198, 353), (265, 437)),
 ((265, 437), (372, 459)),
 ((372, 459), (432, 438)),
 ((432, 438), (492, 373)),
 ((492, 373), (489, 282)),
 ((489, 282), (563, 261)),
 ((563, 261), (525, 187)),
 ((525, 187), (440, 134)),
 ((440, 134), (408, 64)),
 ((408, 64), (298, 139)),
 ((298, 139), (228, 210)),
 ((228, 210), (195, 281)),
 ((195, 281), (208, 396)),
 ((208, 396), (355, 340)),
 ((355, 340), (469, 399)),
 ((469, 399), (515, 370)),
 ((515, 370), (564, 294)),
 ((564, 294), (467, 227)),
 ((467, 227), (298, 262)),
 ((298, 262), (218, 184)),
 ((218, 184), (172, 93)),
 ((172, 93), (88, 64)),
 ((88, 64), (65, 143)),
 ((65, 143

In [19]:
for pontos_andantes in pontos_pareados:

    # 'desempacota' os pares de pontos em variáveis separadas 'ponto_partida_pessoa' e 'ponto_chegada_pessoa'
    ponto_partida_andante = pontos_andantes[0]
    ponto_chegada_andante = pontos_andantes[1]

    # para cada par de pontos_andantes, cria as variáveis 'ai', 'aj', 'bi' e 'bj'
    # para adaptar o código em relação ao trabalho de modelagem 'pdf'
    ai = ponto_partida_andante[0]
    aj = ponto_partida_andante[1]
    bi = ponto_chegada_andante[0]
    bj = ponto_chegada_andante[1]

    # cria a matriz 'ac'
    matriz_ac = np.array([[ai - ci], 
                          [aj - cj]])

    # cria a matriz 'm'    
    matriz_m = np.array(
    [
        [di - ci, ai - bi],
        [dj - cj, aj - bj]
    ]
    )


    try:
        # calcula a inversa da matriz 'm'
        matriz_m_inv = np.linalg.inv(matriz_m)

        # calcula os coeficientes 'k1' e 'k2' do cruzamento
        k2, k1 = matriz_m_inv.dot(matriz_ac)

        if 0 <= k1 <= 1 and 0 <= k2 <= 1:
            print("cruzou")
    except np.linalg.LinAlgError:
        # se a matriz 'm' for singular, não é possível calcular a inversa, ou seja,
        # os vetores são paralelos e não se cruzam
        pass
    
    # cria um frame preto (zeros) com 480 altura X 640 largura pixels e 3 canais de cor (RGB)
    frame = np.zeros((480, 640, 3), dtype=np.uint8)

    # desenha uma seta no frame que representa o vetor de contagem de cruzamentos
    frame = cv2.arrowedLine(img=frame,
                            pt1=pontos_contador["p1"],
                            pt2=pontos_contador["p2"],
                            color=(0, 255, 0),
                            thickness=2)
    
    # desenha uma seta no frame que representa o vetor do objeto 'andante'
    frame = cv2.arrowedLine(img=frame,
                            pt1=ponto_partida_andante,
                            pt2=ponto_chegada_andante,
                            color=(0, 0, 255),
                            thickness=2)
    
    # exibe o frame na tela
    cv2.imshow("Frame", frame)
    
    # congela a execução do código até que uma tecla seja pressionada
    if cv2.waitKey(0) == ord('q'):
        break

# fecha todas as janelas abertas
cv2.destroyAllWindows()


cruzou
cruzou
cruzou
cruzou
cruzou
cruzou
cruzou
cruzou
cruzou
cruzou
cruzou


### Determinando a ``direção`` do cruzamento

In [20]:
# convertendo a nomenclatura para a modelagem dois do trabalho 'pdf'
v1 = vetor_contador
v1

array([  0, 400])

In [21]:
# convertendo a nomenclatura para a modelagem dois do trabalho 'pdf'
v21 = vetor_andante
v21

array([40,  0])

In [22]:
# invertendo o vetor 'andante' (v21) para um vetor 'andante' invertido para checar so o modelo
# que calcula a direção do vetor 'andante' está correto para ambos os lados de cruzamento
v22 = -1 * v21
v22

array([-40,   0])

In [23]:
# v3 é o produto vetorial entre o vetor v1 e o vetor unitário na direção k
v3 = np.array([v1[1], -v1[0]])
v3

array([400,   0])

In [24]:
# aplicando o produto escalar de v3 com o vetor v21 (vetor andante um) (positivo para a direção A->B)
v21.dot(v3)

np.int64(16000)

In [25]:
# aplicando o produto escalar de v3 com o vetor v22 (vetor andante dois) (negativo para a direção B->A)
v22.dot(v3)

np.int64(-16000)

### Repetindo o código anterior e adicionando comandos para determinar a ``direção`` dos cruzamentos

In [26]:
for pontos_andantes in pontos_pareados:
    ponto_partida_pessoa = pontos_andantes[0]
    ponto_chegada_pessoa = pontos_andantes[1]

    # para cada par de pontos_andantes, cria as variáveis 'ai', 'aj', 'bi' e 'bj'
    ai = ponto_partida_pessoa[0]
    aj = ponto_partida_pessoa[1]
    bi = ponto_chegada_pessoa[0]
    bj = ponto_chegada_pessoa[1]

    # cria a matriz 'ac'
    matriz_ac = np.array([[ai - ci], 
                          [aj - cj]])
    
    # cria a matriz 'm'
    matriz_m = np.array(
    [
        [di - ci, ai - bi],
        [dj - cj, aj - bj]
    ]
    )

    try:
        # calcula a inversa da matriz 'm'
        matriz_m_inv = np.linalg.inv(matriz_m)

        # calcula os coeficientes 'k1' e 'k2' do cruzamento
        k2, k1 = matriz_m_inv.dot(matriz_ac)

        if 0 <= k1 <= 1 and 0 <= k2 <= 1:
            vetor_andante = np.array([bi - ai, bj - aj])
            v2 = vetor_andante
            
            produto_escalar = v2.dot(v3)
            if produto_escalar > 0:
                print("cruzou B->A")
            else:
                print("cruzou A->B")

    except np.linalg.LinAlgError:
        pass
    
    # cria um frame preto (zeros) com 480 altura X 640 largura pixels e 3 canais de cor (RGB)
    frame = np.zeros((480, 640, 3), dtype=np.uint8)

    # desenha uma seta no frame que representa o vetor de contagem de cruzamentos
    frame = cv2.arrowedLine(img=frame,
                            pt1=pontos_contador["p1"],
                            pt2=pontos_contador["p2"],
                            color=(0, 255, 0),
                            thickness=2)
    
    # desenha uma seta no frame que representa o vetor do objeto 'andante'
    frame = cv2.arrowedLine(img=frame,
                            pt1=ponto_partida_pessoa,
                            pt2=ponto_chegada_pessoa,
                            color=(0, 0, 255),
                            thickness=2)
    
    # exibe o frame na tela
    cv2.imshow("Frame", frame)
    
    # congela a execução do código até que uma tecla seja pressionada
    if cv2.waitKey(0) == ord('q'):
        break

# fecha todas as janelas abertas
cv2.destroyAllWindows()

cruzou A->B
cruzou B->A
cruzou A->B
cruzou B->A
cruzou A->B
cruzou B->A
cruzou A->B
cruzou B->A
cruzou A->B
cruzou B->A
cruzou A->B
